In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
import torch 
import foolbox
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cuda:0" )
print(device)

cuda:0


In [2]:
def cast_float(x):
    return x.astype(np.float32)

def cast_int(x):
    return x.astype(np.int32)

def create_partition(x_orig_train, y_orig_train, p_train=0.2):
  # inspired by pberkes answer: https://stackoverflow.com/questions/3674409/how-to-split-partition-a-dataset-into-training-and-test-datasets-for-e-g-cros, 
  # seed so we always get the same partition (can be changed later)
    np.random.seed(0)

  # generate random indices
    random_indices = np.random.permutation(x_orig_train.shape[0])

  # calculate how much to put in each partition
    test_size = int(x_orig_train.shape[0] * p_train)

  # split up the training and testing data in the same way
    testing_indices = random_indices[:test_size] # all before test_size
    training_indices = random_indices[test_size:] # all after test_size

    x_train, y_train = x_orig_train[training_indices, :], y_orig_train[training_indices]
    x_test, y_test = x_orig_train[testing_indices, :], y_orig_train[testing_indices]

    return x_train, y_train, x_test, y_test

def get_pdfrate(test=False):
    train_data = datasets.load_svmlight_file("pdf_dataset/data/pdfrateB_train.libsvm", n_features=135, zero_based=True)
    x_orig_train, y_orig_train = train_data[0].toarray(), train_data[1]

    x_train, y_train, x_test, y_test = create_partition(x_orig_train, y_orig_train)

    if test:
        test_data = datasets.load_svmlight_file("pdf_dataset/data/pdfrateB_test.libsvm", n_features=135, zero_based=True)
        x_test, y_test = test_data[0].toarray(), test_data[1]

    x_train = 1 - 2*x_train
    x_test = 1 - 2*x_test
  
    return cast_float(x_train), cast_int(y_train), cast_float(x_test), cast_int(y_test)


In [3]:
X_train, Y_train, X_test, Y_test = get_pdfrate(test=True)
X_train = torch.from_numpy(X_train)
Y_train = torch.from_numpy(Y_train).long()
X_test = torch.from_numpy(X_test)
Y_test = torch.from_numpy(Y_test).long()

In [4]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

torch.Size([8066, 135])
torch.Size([8066])
torch.Size([9771, 135])


In [5]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SubsetRandomSampler, TensorDataset

In [6]:
Training_set = TensorDataset(X_train, Y_train)
Testing_set = TensorDataset(X_test, Y_test)

In [7]:
def scaled_sigmoid(input):
    return 2*torch.sigmoid(input)-1

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(135, 1024)
        self.fc2 = nn.Linear(1024, 2)

    def forward(self, x):
        
        x = scaled_sigmoid(self.fc1(x))
        x = F.softmax(self.fc2(x),dim=1)
        return x

In [8]:
def train(model,trainloader,testloader,Epoch):
    train_acc = []
    test_acc = []
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
#     train_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=1)
    for epoch in range(Epoch):  # loop over the dataset multiple times
        PATH = "models/pdfrate/clean_model_epoch_{}.pt".format(epoch)
        print("Epoch,",epoch+1)
        model.train()
#         train_lr_scheduler.step()
        running_loss = 0.0
        correct = 0
        toatl = 0 
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device),data[1].to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            pred=outputs.argmax(dim=1 , keepdim=True)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            correct+=pred.eq(labels.view_as(pred)).sum().item()
            
        print("Training accuarcy equals:",correct/len(trainloader.dataset))
        print("Loss equals:",running_loss)
        model.eval()
        correct = 0
        for i, data in enumerate(testloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device),data[1].to(device)
            outputs = model(inputs) 
            pred=outputs.argmax(dim=1 , keepdim=True)
            correct+=pred.eq(labels.view_as(pred)).sum().item()      
        print("Testing accuarcy equals:",correct/len(testloader.dataset))
        torch.save(model.state_dict(),PATH)
    print('Finished Training')

In [9]:
model1 = Net()
Train_loader = DataLoader(Training_set, batch_size=512,shuffle=True, num_workers=2)
Test_loader = DataLoader(Testing_set, batch_size=512,shuffle=True, num_workers=2)

In [10]:
CLEANMODEL_PATH = "models/pdfrate/clean/clean_model_epoch_99.pt"
try:
    model1.load_state_dict(torch.load(CLEANMODEL_PATH))
    print('Get the saved model')
except IOError:
    print('No model in the dirctory, get a pretrained vgg16 trained on imaigenet.')
    print("Start Training")
    train(model1,Train_loader,Test_loader,100)
    print("Training complete")
    torch.save(model1.state_dict(),CLEANMODEL_PATH)

Get the saved model


In [11]:
def stabilize_l1(model):
    old_weights = model.fc1.weight.t()
#     print(old_weights)
#     print(old_weights.shape)
    new_weights = torch.sign(old_weights)
    old_mags = torch.linalg.norm(old_weights,ord = float('inf'),dim = 0)
    scale_matrix = torch.diag(old_mags)
#     print(scale_matrix.shape)
    scale_weights = torch.matmul(new_weights,scale_matrix).t()
    with torch.no_grad():
        model.fc1.weight = torch.nn.Parameter(scale_weights)
    for params in model.parameters():
        params.requires_grad = False 
        break 

    return model 

In [12]:
def evaluate(model,testloader):
    model.eval()
    correct = 0
    model.to(device)
    for i, data in enumerate(testloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device),data[1].to(device)
        outputs = model(inputs) 
        pred=outputs.argmax(dim=1 , keepdim=True)
        correct+=pred.eq(labels.view_as(pred)).sum().item()      
    print("Testing accuarcy equals:",correct/len(testloader.dataset))
    return correct/len(testloader.dataset)

In [13]:
def bbattack(model,inputs,labels):
    brendel_attack = foolbox.attacks.L1BrendelBethgeAttack(steps=20)
    foolbox_model = foolbox.models.PyTorchModel(model.eval(), bounds=(-2, 2))
    criterion = foolbox.criteria.Misclassification(labels)
    advvs = None
    while advvs is None:
        try:
            advvs, _, _ = brendel_attack(foolbox_model, inputs, criterion, epsilons=None)
        except:
            pass 
    return advvs

def Adv_Stabilize_train(model,trainloader,testloader,epoch1,epoch2,epsilon):
    test_acc = []
    model.to(device)
    criter = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    for oe in range(epoch1,epoch2):
        print("epoch",oe)
        PATH = "models/pdfrate/adv/constriant_adv_{}_model_epoch_{}.pt".format(epsilon,oe)
        running_loss = 0 
        correct = 0
        for i, data in enumerate(trainloader, 0):
    # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device),data[1].to(device)
            advs = bbattack(model,inputs,labels)
            optimizer.zero_grad()
            diff = torch.abs(advs-inputs).detach()
            l1flag = torch.sum(diff,1)
            length = len(l1flag)
            for j in range(length):
                if l1flag[j]>epsilon:
                    advs[j] = inputs[j]
            outputs = model(advs)
            model.train()
            loss = criter(outputs, labels)
            pred=outputs.argmax(dim=1 , keepdim=True)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            correct+=pred.eq(labels.view_as(pred)).sum().item()
        print("Training accuarcy equals:",correct/len(trainloader.dataset))
        print("Loss equals:",running_loss)
        
        model.eval()
        correct = 0
        for i, data in enumerate(testloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device),data[1].to(device)
            outputs = model(inputs) 
            pred=outputs.argmax(dim=1 , keepdim=True)
            correct+=pred.eq(labels.view_as(pred)).sum().item()      
        print("Testing accuarcy equals:",correct/len(testloader.dataset))
        test_acc.append(correct/len(testloader.dataset))


        torch.save(model.state_dict(),PATH)
        
    return test_acc

In [14]:
Test_acc , Test_robust = Adv_Stabilize_train(model1,Train_loader,Test_loader,0,15,50)

epoch 0
Training accuarcy equals: 0.0007438631291842301
Loss equals: 12.283328771591187
Testing accuarcy equals: 0.6612424521543343
norm: tensor(19.8202, device='cuda:0')
Testing robustness equals: 0.0
epoch 1


/home/thisismichael/anaconda3/lib/python3.7/site-packages/eagerpy/tensor/pytorch.py:257: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return type(self)(torch.as_tensor(a, device=self.raw.device))


Training accuarcy equals: 0.3730473592858914
Loss equals: 12.835405468940735
Testing accuarcy equals: 0.5475386347354416
norm: tensor(26.8440, device='cuda:0')
Testing robustness equals: 0.5191894381332515
epoch 2
Training accuarcy equals: 0.5388048599057773
Loss equals: 12.213464200496674
Testing accuarcy equals: 0.5422167638931532
norm: tensor(26.9216, device='cuda:0')
Testing robustness equals: 0.5298331798178283
epoch 3
Training accuarcy equals: 0.544011901810067
Loss equals: 12.210780382156372
Testing accuarcy equals: 0.5409886398526251
norm: tensor(26.9443, device='cuda:0')
Testing robustness equals: 0.5340292702896325
epoch 4
Training accuarcy equals: 0.5473592858913959
Loss equals: 12.202835500240326
Testing accuarcy equals: 0.5406816088424931
norm: tensor(26.9554, device='cuda:0')
Testing robustness equals: 0.5359738000204687
epoch 5
Training accuarcy equals: 0.5484750805851724
Loss equals: 12.191369593143463
Testing accuarcy equals: 0.540579265172449
norm: tensor(26.9608, dev

In [15]:
model_test = Net()
Test_acc =[]
Test_robust = []
epsilon = 50
for it in range(15):
    PATH = "models/pdfrate/adv/constriant_adv_50_model_epoch_{}.pt".format(it)
    model_test.load_state_dict(torch.load(PATH))
    clean_acc = evaluate(model_test,Test_loader)
    Test_acc.append(clean_acc)
    correct = 0 
    for i, data in enumerate(Test_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device),data[1].to(device)
        advs = bbattack(model_test,inputs,labels)
        diff = torch.abs(advs-inputs).detach()
        l1flag = torch.sum(diff,1)
        length = len(l1flag)
        for j in range(length):
            if l1flag[j]>epsilon:
                advs[j] = inputs[j]
        outputs = model_test(advs) 
        pred = outputs.argmax(dim=1 , keepdim=True)
        correct += pred.eq(labels.view_as(pred)).sum().item()
    robust = correct/len(Test_loader.dataset)
    Test_robust.append(robust)
    print("Robustness:",robust)

Testing accuarcy equals: 0.6612424521543343
Robustness: 0.00010234367004400778
Testing accuarcy equals: 0.5475386347354416
Robustness: 0.5197011564834715
Testing accuarcy equals: 0.5422167638931532
Robustness: 0.5297308361477843
Testing accuarcy equals: 0.5409886398526251
Robustness: 0.5340292702896325
Testing accuarcy equals: 0.5406816088424931
Robustness: 0.5359738000204687
Testing accuarcy equals: 0.540579265172449


RuntimeError: DataLoader worker (pid(s) 16872, 16873) exited unexpectedly

In [ ]:
model_test = Net()
Test_acc_sd =[]
Test_robust_sd = []
for it in range(15):
    PATH = "models/pdfrate/adv/constriant_adv_50_model_epoch_{}.pt".format(it)
    SD_PATH = "models/pdfrate/stable/stable_constriant_adv_50_model_epoch_{}.pt".format(it)
    model_test.load_state_dict(torch.load(PATH))
    model_sd = stabilize_l1(model_test)
    torch.save(model_sd.state_dict(),SD_PATH)
    clean_acc = evaluate(model_sd,Test_loader)
    Test_acc_sd.append(clean_acc)
    correct = 0 
    for i, data in enumerate(Test_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device),data[1].to(device)
        advs = bbattack(model_sd,inputs,labels)
        diff = torch.abs(advs-inputs).detach()
        l1flag = torch.sum(diff,1)
        length = len(l1flag)
        for j in range(length):
            if l1flag[j]>epsilon:
                advs[j] = inputs[j]
        outputs = model_sd(advs) 
        pred = outputs.argmax(dim=1 , keepdim=True)
        correct += pred.eq(labels.view_as(pred)).sum().item()
    robust = correct/len(Test_loader.dataset)
    Test_robust_sd.append(robust)
    print("Robustness:",robust)

In [ ]:
X = torch.linspace(1,15,15)
Y1 = Test_robust
Y2 = Test_robust_sd
plt.plot(X,Y1,label='Adversarial training')
plt.plot(X,Y2,label='Stabilization after Adversarial')
plt.ylabel('Robustness')
plt.xlabel('Epochs of Adversarial Training')
plt.title("Robustness under L1 BrendalBethge attack less than 30")
plt.legend()
plt.savefig("RobustRatioeps30.jpg")
plt.show

In [ ]:
Y1 = Test_acc
Y2 = Test_acc_sd
plt.plot(X,Y1,label='Adversarial training')
plt.plot(X,Y2,label='Stabilization after Adversarial')
plt.ylabel('Accuarcy')
plt.xlabel('Epochs of Adversarial Training')
plt.title("Test Accuarcy ")
plt.legend()
plt.savefig("AccRatioeps30.jpg")
plt.show

In [ ]:
model3 = stabilize()